<a href="https://colab.research.google.com/github/mahdimalverdi/Bigdata-hw5/blob/main/install_on_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

In [3]:
import findspark
findspark.init()

In [13]:
from google.colab import files
files.upload()

Saving iris_test.csv to iris_test (1).csv


{'iris_test.csv': b'SepalLength,SepalWidth,PetalLength,PetalWidth,Species\r\n5.1,3.5,1.4,0.2,setosa\r\n4.9,3,1.4,0.2,setosa\r\n4.6,3.1,1.5,0.2,setosa\r\n5,3.4,1.5,0.2,setosa\r\n4.4,2.9,1.4,0.2,setosa\r\n4.3,3,1.1,0.1,setosa\r\n5.1,3.8,1.5,0.3,setosa\r\n5,3,1.6,0.2,setosa\r\n4.8,3.1,1.6,0.2,setosa\r\n5.5,3.5,1.3,0.2,setosa\r\n4.4,3,1.3,0.2,setosa\r\n5.1,3.4,1.5,0.2,setosa\r\n4.5,2.3,1.3,0.3,setosa\r\n5,3.5,1.6,0.6,setosa\r\n5.1,3.8,1.6,0.2,setosa\r\n7,3.2,4.7,1.4,versicolor\r\n6.4,3.2,4.5,1.5,versicolor\r\n6.9,3.1,4.9,1.5,versicolor\r\n5,2,3.5,1,versicolor\r\n6,2.2,4,1,versicolor\r\n6.1,2.9,4.7,1.4,versicolor\r\n5.8,2.7,4.1,1,versicolor\r\n5.9,3.2,4.8,1.8,versicolor\r\n6.1,2.8,4,1.3,versicolor\r\n6.1,2.8,4.7,1.2,versicolor\r\n6.6,3,4.4,1.4,versicolor\r\n6.8,2.8,4.8,1.4,versicolor\r\n6.7,3,5,1.7,versicolor\r\n6,2.9,4.5,1.5,versicolor\r\n5.5,2.5,4,1.3,versicolor\r\n6.5,3,5.8,2.2,virginica\r\n7.6,3,6.6,2.1,virginica\r\n6.4,2.7,5.3,1.9,virginica\r\n5.7,2.5,5,2,virginica\r\n5.8,2.8,5.1,2.4,v

In [62]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DoubleType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType
from pyspark.sql.functions import col,array_contains
spark = SparkSession.builder.master("local[1]").appName("hw5").getOrCreate()
train_dataset = spark.read.csv("iris_train.csv",inferSchema=True, header =True)
train_dataset.printSchema()
test_dataset = spark.read.csv("iris_test.csv",inferSchema=True, header =True)
test_dataset = test_dataset.withColumnRenamed("SepalLength","Sepal_Length")
test_dataset = test_dataset.withColumnRenamed("SepalWidth","Sepal_Width")
test_dataset = test_dataset.withColumnRenamed("PetalLength","Petal_Length")
test_dataset = test_dataset.withColumnRenamed("PetalWidth","Petal_Width")
test_dataset = test_dataset.withColumnRenamed("Species","class")
test_dataset.printSchema()

root
 |-- Sepal_Length: double (nullable = true)
 |-- Sepal_Width: double (nullable = true)
 |-- Petal_Length: double (nullable = true)
 |-- Petal_Width: double (nullable = true)
 |-- class: string (nullable = true)

root
 |-- Sepal_Length: double (nullable = true)
 |-- Sepal_Width: double (nullable = true)
 |-- Petal_Length: double (nullable = true)
 |-- Petal_Width: double (nullable = true)
 |-- class: string (nullable = true)



In [89]:
from pyspark.ml.feature import VectorAssembler
import pyspark.sql.functions as F

assembler = VectorAssembler(inputCols=['Sepal_Length', 'Sepal_Width', 'Petal_Length', 'Petal_Width'], outputCol = 'Attributes')
mapping= {
      'setosa': 1.0,
      'versicolor': 2.0,
      'virginica': 3.0
  }
def mapper(value):
  return mapping[value]

udfmapper = F.udf(mapper, DoubleType())

def finalize(data):
  data = data.withColumn('number_class',udfmapper('class'))
  output = assembler.transform(data)
  finalized_data = output.select("Attributes","number_class")
  finalized_data.show()
  return finalized_data

train_data = finalize(train_dataset)
test_data = finalize(test_dataset)

+-----------------+------------+
|       Attributes|number_class|
+-----------------+------------+
|[4.7,3.2,1.3,0.2]|         1.0|
|[5.0,3.6,1.4,0.2]|         1.0|
|[5.4,3.9,1.7,0.4]|         1.0|
|[4.6,3.4,1.4,0.3]|         1.0|
|[4.9,3.1,1.5,0.1]|         1.0|
|[5.4,3.7,1.5,0.2]|         1.0|
|[4.8,3.4,1.6,0.2]|         1.0|
|[4.8,3.0,1.4,0.1]|         1.0|
|[5.8,4.0,1.2,0.2]|         1.0|
|[5.7,4.4,1.5,0.4]|         1.0|
|[5.4,3.9,1.3,0.4]|         1.0|
|[5.1,3.5,1.4,0.3]|         1.0|
|[5.7,3.8,1.7,0.3]|         1.0|
|[5.4,3.4,1.7,0.2]|         1.0|
|[5.1,3.7,1.5,0.4]|         1.0|
|[4.6,3.6,1.0,0.2]|         1.0|
|[5.1,3.3,1.7,0.5]|         1.0|
|[4.8,3.4,1.9,0.2]|         1.0|
|[5.0,3.4,1.6,0.4]|         1.0|
|[5.2,3.5,1.5,0.2]|         1.0|
+-----------------+------------+
only showing top 20 rows

+-----------------+------------+
|       Attributes|number_class|
+-----------------+------------+
|[5.1,3.5,1.4,0.2]|         1.0|
|[4.9,3.0,1.4,0.2]|         1.0|
|[4.6,3.1,1.5,0.2

In [130]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import RegressionEvaluator

def evaluate(result):
  evaluator = MulticlassClassificationEvaluator(
      labelCol="number_class", predictionCol="prediction", metricName="accuracy")
  accuracy = evaluator.evaluate(result)
  print("Test Error = %g " % (1.0 - accuracy))

In [161]:
from pyspark.ml.regression import DecisionTreeRegressor
regressor = DecisionTreeRegressor(featuresCol = 'Attributes', labelCol = 'number_class')
regressor = regressor.fit(train_data)
result = regressor.transform(test_data)
result.show()
evaluate(result)

+-----------------+------------+----------+
|       Attributes|number_class|prediction|
+-----------------+------------+----------+
|[5.1,3.5,1.4,0.2]|         1.0|       1.0|
|[4.9,3.0,1.4,0.2]|         1.0|       1.0|
|[4.6,3.1,1.5,0.2]|         1.0|       1.0|
|[5.0,3.4,1.5,0.2]|         1.0|       1.0|
|[4.4,2.9,1.4,0.2]|         1.0|       1.0|
|[4.3,3.0,1.1,0.1]|         1.0|       1.0|
|[5.1,3.8,1.5,0.3]|         1.0|       1.0|
|[5.0,3.0,1.6,0.2]|         1.0|       1.0|
|[4.8,3.1,1.6,0.2]|         1.0|       1.0|
|[5.5,3.5,1.3,0.2]|         1.0|       1.0|
|[4.4,3.0,1.3,0.2]|         1.0|       1.0|
|[5.1,3.4,1.5,0.2]|         1.0|       1.0|
|[4.5,2.3,1.3,0.3]|         1.0|       1.0|
|[5.0,3.5,1.6,0.6]|         1.0|       1.0|
|[5.1,3.8,1.6,0.2]|         1.0|       1.0|
|[7.0,3.2,4.7,1.4]|         2.0|       2.0|
|[6.4,3.2,4.5,1.5]|         2.0|       2.0|
|[6.9,3.1,4.9,1.5]|         2.0|       2.0|
|[5.0,2.0,3.5,1.0]|         2.0|       2.0|
|[6.0,2.2,4.0,1.0]|         2.0|

In [132]:
from pyspark.ml.regression import LinearRegression
from pyspark.sql.functions import round 
regressor = LinearRegression(featuresCol = 'Attributes', labelCol = 'number_class')
regressor = regressor.fit(train_data)
pred = regressor.evaluate(test_data)
result = pred.predictions
result = result.withColumn('prediction',round(col('prediction')))
result.show()
evaluate(result)

+-----------------+------------+----------+
|       Attributes|number_class|prediction|
+-----------------+------------+----------+
|[5.1,3.5,1.4,0.2]|         1.0|       1.0|
|[4.9,3.0,1.4,0.2]|         1.0|       1.0|
|[4.6,3.1,1.5,0.2]|         1.0|       1.0|
|[5.0,3.4,1.5,0.2]|         1.0|       1.0|
|[4.4,2.9,1.4,0.2]|         1.0|       1.0|
|[4.3,3.0,1.1,0.1]|         1.0|       1.0|
|[5.1,3.8,1.5,0.3]|         1.0|       1.0|
|[5.0,3.0,1.6,0.2]|         1.0|       1.0|
|[4.8,3.1,1.6,0.2]|         1.0|       1.0|
|[5.5,3.5,1.3,0.2]|         1.0|       1.0|
|[4.4,3.0,1.3,0.2]|         1.0|       1.0|
|[5.1,3.4,1.5,0.2]|         1.0|       1.0|
|[4.5,2.3,1.3,0.3]|         1.0|       1.0|
|[5.0,3.5,1.6,0.6]|         1.0|       1.0|
|[5.1,3.8,1.6,0.2]|         1.0|       1.0|
|[7.0,3.2,4.7,1.4]|         2.0|       2.0|
|[6.4,3.2,4.5,1.5]|         2.0|       2.0|
|[6.9,3.1,4.9,1.5]|         2.0|       2.0|
|[5.0,2.0,3.5,1.0]|         2.0|       2.0|
|[6.0,2.2,4.0,1.0]|         2.0|

In [133]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes(featuresCol = 'Attributes', labelCol = 'number_class')
model = nb.fit(train_data)
result = model.transform(test_data)
result = result.withColumn("prediction", F.col("prediction")+1)
result.show()
evaluate(result)

+-----------------+------------+--------------------+--------------------+----------+
|       Attributes|number_class|       rawPrediction|         probability|prediction|
+-----------------+------------+--------------------+--------------------+----------+
|[5.1,3.5,1.4,0.2]|         1.0|[-11.957946688251...|[0.74996852808446...|       1.0|
|[4.9,3.0,1.4,0.2]|         1.0|[-11.275671914190...|[0.68189370315239...|       1.0|
|[4.6,3.1,1.5,0.2]|         1.0|[-11.362896074031...|[0.66531523853579...|       1.0|
|[5.0,3.4,1.5,0.2]|         1.0|[-11.972470666290...|[0.71990404708011...|       1.0|
|[4.4,2.9,1.4,0.2]|         1.0|[-10.810300977956...|[0.64663407333519...|       1.0|
|[4.3,3.0,1.1,0.1]|         1.0|[-9.9032839209271...|[0.73209396911236...|       1.0|
|[5.1,3.8,1.5,0.3]|         1.0|[-12.837112496781...|[0.74404024056896...|       1.0|
|[5.0,3.0,1.6,0.2]|         1.0|[-11.734937423182...|[0.65266678729196...|       1.0|
|[4.8,3.1,1.6,0.2]|         1.0|[-11.699784039860...|[

In [162]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol = 'Attributes', labelCol = 'number_class')
lrModel = lr.fit(train_data)
result = lrModel.transform(test_data)
result.show()
evaluate(result)


+-----------------+------------+--------------------+--------------------+----------+
|       Attributes|number_class|       rawPrediction|         probability|prediction|
+-----------------+------------+--------------------+--------------------+----------+
|[5.1,3.5,1.4,0.2]|         1.0|[-54.090809199812...|[7.08626592898545...|       1.0|
|[4.9,3.0,1.4,0.2]|         1.0|[-49.458527644804...|[4.10093110607260...|       1.0|
|[4.6,3.1,1.5,0.2]|         1.0|[-48.844913922307...|[1.91062986208791...|       1.0|
|[5.0,3.4,1.5,0.2]|         1.0|[-52.942459836901...|[5.11360811774894...|       1.0|
|[4.4,2.9,1.4,0.2]|         1.0|[-46.366494864048...|[2.41449528159736...|       1.0|
|[4.3,3.0,1.1,0.1]|         1.0|[-46.346577425706...|[2.58351706780992...|       1.0|
|[5.1,3.8,1.5,0.3]|         1.0|[-56.471383510962...|[3.36423165461291...|       1.0|
|[5.0,3.0,1.6,0.2]|         1.0|[-50.050452026521...|[1.19409793314021...|       1.0|
|[4.8,3.1,1.6,0.2]|         1.0|[-49.847042353302...|[